In [1]:
import numpy as np
import pandas as pd

In [16]:
# Wir gehen ins richtige Arbeitsverzeichnis, dorthin, wo unsere Covid Datenbank liegt.

%cd ~/Documents/Work/Tutorials/RKI_Covid

covid_path = "211203_RKI_COVID19.csv"

#Achtung: Im Github ist nur ein kleiner Auszug der Original RKI Covid_Datei vom 03.12.21 abgelegt.
# Hinweis zu den Daten (Stand 03.12.2021):
# Die Daten sind die „Fallzahlen in Deutschland“ des Robert Koch-Institut (RKI) und stehen unter der Open Data Datenlizenz Deutschland – Namensnennung – Version 2.0 zur Verfügung. 
# Quellenvermerk: Robert Koch-Institut (RKI), dl-de/by-2-0

/Users/uli/Documents/Work/Tutorials/RKI_Covid


In [17]:
# Covid19 Datenbank einlesen und dem Datafram covid zuweisen. covid enthält nun die Daten
covid = pd.read_csv(covid_path)

In [18]:
# Schauen wir uns nur die drei Zeilen des dataframes covid an. 
# Dazu verwenden wir die Funktion head(3) und wenden diese auf covid an.
# Eine Spalte enthält das jeweilige Meldedatum

covid.head(3)

,ObjectId,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
0,1,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,2,0,2021/11/17 00:00:00+00,1001,"02.12.2021, 00:00 Uhr",0,-9,2021/11/12 00:00:00+00,0,2,1,Nicht übermittelt
1,2,1,Schleswig-Holstein,SK Flensburg,A15-A34,W,1,0,2021/09/27 00:00:00+00,1001,"02.12.2021, 00:00 Uhr",0,-9,2021/09/19 00:00:00+00,0,1,1,Nicht übermittelt
2,3,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,3,0,2021/11/17 00:00:00+00,1001,"02.12.2021, 00:00 Uhr",0,-9,2021/11/14 00:00:00+00,0,3,1,Nicht übermittelt


In [20]:
# Pandas kann mit Datumswerten umgehen, hat Pandas diesen Datenwert richtig erkannt?
# Wir überprüfen das mit
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3034186 entries, 0 to 3034185
Data columns (total 18 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   ObjectId              int64 
 1   IdBundesland          int64 
 2   Bundesland            object
 3   Landkreis             object
 4   Altersgruppe          object
 5   Geschlecht            object
 6   AnzahlFall            int64 
 7   AnzahlTodesfall       int64 
 8   Meldedatum            object
 9   IdLandkreis           int64 
 10  Datenstand            object
 11  NeuerFall             int64 
 12  NeuerTodesfall        int64 
 13  Refdatum              object
 14  NeuGenesen            int64 
 15  AnzahlGenesen         int64 
 16  IstErkrankungsbeginn  int64 
 17  Altersgruppe2         object
dtypes: int64(10), object(8)
memory usage: 416.7+ MB


In [21]:
# Nein, leider wurde beim einlesen nicht erkannt, dass der Datentyp
# des Meldedatums ein Datum ist.
# Zum Glück bietet Pandas uns hier ein Werkzeug an: Wir lesen die Datenbank
# jetzt einfach noch einmal neu und wir sagen Pandas beim Einlesen
# der Daten, dass Die Spalte Meldedatum ein Datumsformat enthält. Dies machen
# wir mit der Option parse_dates, was soviel heisst wie 
# Interpretiere die Werte von Meldedatum als Datumswerte

covid = pd.read_csv(covid_path, parse_dates=['Meldedatum'])

In [22]:
# Jetzt machen wir uns an die erste Aufgabe. Wir wollen eine Tabelle erstellen, die
# die für jeden Tag für jeden Landkreis alle gemeldeten Fälle enthält.
# Dabei kennzeichnet das datum die Zeile und die Landkreise sind die einzelnen Spalten

In [23]:
# wir brauchen also erst einmal ein paar Eckdaten:
# An welchem Tag beginnt unsere Datenbank und was ist das letzte Datum?
# Und welche Landkreise haben wir eigentlich?

#Für Frage 1 schauen wir uns die Spalte Meldedatum an:

start=covid['Meldedatum'].min()
ende=covid['Meldedatum'].max()

# Die Landkreise erhalten wir aus der Spalte 'Landkreis' und 
# wenden darauf die Funktion unique an. Unique listet als Ergebnis nur 
# die Werte werte auf die unterschiedlich sind, also doppelte Werte werden
# nur einmal aufgelistet. Das Ergebnis kommt in die Variable landkreise
# Die Budesländer nehmen wir anlaog auch gleich mit und speichern das Ergebnis in
# der Variablen bundesländer

landkreise=covid['Landkreis'].unique()
bundeslaender=covid['Bundesland'].unique()

In [24]:
# So, was enthalten denn nun unsere Variablen start, ende, landkreise und bundesländer?
start

Timestamp('2020-01-02 00:00:00+0000', tz='UTC')

In [25]:
ende

Timestamp('2021-12-01 00:00:00+0000', tz='UTC')

In [26]:
# Das sieht fast ein wenig kryptisch aus, ist aber noch lesbar.
# Gespeichert sind hier die Daten im pandas internen datumsformat.
# Man erkennt sehr schön das Datum und die Zeit, eine Zeitzone ist ebenfalls mit angegeben.

In [27]:
# landkreise enthält als eindimensionales array ale Landkreisnamen
landkreise

array(['SK Flensburg', 'SK Kiel', 'SK Lübeck', 'SK Neumünster',
       'LK Dithmarschen', 'LK Herzogtum Lauenburg', 'LK Nordfriesland',
       'LK Ostholstein', 'LK Pinneberg', 'LK Plön',
       'LK Rendsburg-Eckernförde', 'LK Schleswig-Flensburg',
       'LK Segeberg', 'LK Steinburg', 'LK Stormarn', 'SK Hamburg',
       'SK Braunschweig', 'SK Salzgitter', 'SK Wolfsburg', 'LK Gifhorn',
       'LK Goslar', 'LK Helmstedt', 'LK Northeim', 'LK Peine',
       'LK Wolfenbüttel', 'LK Göttingen', 'Region Hannover',
       'LK Diepholz', 'LK Hameln-Pyrmont', 'LK Hildesheim',
       'LK Holzminden', 'LK Nienburg (Weser)', 'LK Schaumburg',
       'LK Celle', 'LK Cuxhaven', 'LK Harburg', 'LK Lüneburg',
       'LK Lüchow-Dannenberg', 'LK Osterholz', 'LK Rotenburg (Wümme)',
       'LK Heidekreis', 'LK Stade', 'LK Uelzen', 'LK Verden',
       'SK Delmenhorst', 'SK Emden', 'SK Oldenburg', 'SK Osnabrück',
       'SK Wilhelmshaven', 'LK Ammerland', 'LK Aurich', 'LK Cloppenburg',
       'LK Emsland', 'LK

In [28]:
# bundeslaender ist ebenfalls ein eindimensionales Array
bundeslaender

array(['Schleswig-Holstein', 'Hamburg', 'Niedersachsen', 'Bremen',
       'Nordrhein-Westfalen', 'Hessen', 'Rheinland-Pfalz',
       'Baden-Württemberg', 'Bayern', 'Saarland', 'Berlin', 'Brandenburg',
       'Mecklenburg-Vorpommern', 'Sachsen', 'Sachsen-Anhalt', 'Thüringen'],
      dtype=object)

In [29]:
print('Start Erfassung:', start.strftime('%d.%m.%Y')) 
print('Ende  Erfassung:', ende.strftime('%d.%m.%Y'))
print('Anzahl Bundesländer:', len(bundeslaender))
print('Anzahl Landkreise:', len(landkreise))

Start Erfassung: 02.01.2020
Ende  Erfassung: 01.12.2021
Anzahl Bundesländer: 16
Anzahl Landkreise: 411


In [43]:
# Und jetzt kommen wir zum kleinen Höhepunkt unseres Tutorials.
# Wir stellen jetzt unseren Datafram um, indem wir nach Landkreis und Datum
# gruppieren. Das geht einfach mit dem Befehl groupby. Gleichzeitig sollen die Spalten
# alle Zeilen, die gleiche Landkreise enthalten addiert werden. Dazu stellen 
# wir die Funktion sum() ans Ende.

cov_gruppiert = covid.groupby(['Landkreis', 'Meldedatum']).sum()

cov_gruppiert

ObjectId  IdBundesland  \
Landkreis           Meldedatum                                          
LK Ahrweiler        2020-03-12 00:00:00+00:00   5922237            35   
                    2020-03-13 00:00:00+00:00   2367136            14   
                    2020-03-14 00:00:00+00:00   1186555             7   
                    2020-03-16 00:00:00+00:00   3555088            21   
                    2020-03-17 00:00:00+00:00   5925148            35   
...                                                 ...           ...   
StädteRegion Aachen 2021-11-27 00:00:00+00:00  37470566           320   
                    2021-11-28 00:00:00+00:00  12248676           105   
                    2021-11-29 00:00:00+00:00  23337852           200   
                    2021-11-30 00:00:00+00:00  23912086           205   
                    2021-12-01 00:00:00+00:00  13421400           115   

                                               AnzahlFall  AnzahlTodesfall  \
Landkreis           Meldedatum                                               
LK Ahrweiler        2020-03-12 00:00:00+00:00           6                0   
                    2020-03-13 00:00:00+00:00           3                0   
                    2020-03-14 00:00:00+00:00           1                0   
                    2020-03-16 00:00:00+00:00           4                0   
                    2020-03-17 00:00:00+00:00           6                0   
...                                                   ...              ...   
StädteRegion Aachen 2021-11-27 00:00:00+00:00         205                0   
                    2021-11-28 00:00:00+00:00          51                0   
                    2021-11-29 00:00:00+00:00         206                0   
                    2021-11-30 00:00:00+00:00         355                0   
                    2021-12-01 00:00:00+00:00          45                0   

                                               IdLandkreis  NeuerFall  \
Landkreis           Meldedatum                                          
LK Ahrweiler        2020-03-12 00:00:00+00:00        35655          0   
                    2020-03-13 00:00:00+00:00        14262          0   
                    2020-03-14 00:00:00+00:00         7131          0   
                    2020-03-16 00:00:00+00:00        21393          0   
                    2020-03-17 00:00:00+00:00        35655          0   
...                                                    ...        ...   
StädteRegion Aachen 2021-11-27 00:00:00+00:00       341376          0   
                    2021-11-28 00:00:00+00:00       112014          0   
                    2021-11-29 00:00:00+00:00       213360          2   
                    2021-11-30 00:00:00+00:00       218694         24   
                    2021-12-01 00:00:00+00:00       122682         23   

                                               NeuerTodesfall  NeuGenesen  \
Landkreis           Meldedatum                                              
LK Ahrweiler        2020-03-12 00:00:00+00:00             -45           0   
                    2020-03-13 00:00:00+00:00             -18           0   
                    2020-03-14 00:00:00+00:00              -9           0   
                    2020-03-16 00:00:00+00:00             -27           0   
                    2020-03-17 00:00:00+00:00             -45           0   
...                                                       ...         ...   
StädteRegion Aachen 2021-11-27 00:00:00+00:00            -576        -539   
                    2021-11-28 00:00:00+00:00            -189        -179   
                    2021-11-29 00:00:00+00:00            -360        -342   
                    2021-11-30 00:00:00+00:00            -369        -332   
                    2021-12-01 00:00:00+00:00            -207        -177   

                                               AnzahlGenesen  \
Landkreis           Meldedatum                                 
LK

In [44]:
# Schauen wir uns an, wie wir hier einzelne Landkreise herausgreifen können

cov_gruppiert.loc['SK Duisburg','AnzahlFall']

Meldedatum
2020-03-01 00:00:00+00:00      1
2020-03-04 00:00:00+00:00      1
2020-03-06 00:00:00+00:00      1
2020-03-10 00:00:00+00:00      5
2020-03-11 00:00:00+00:00      1
                            ... 
2021-11-27 00:00:00+00:00    150
2021-11-28 00:00:00+00:00    121
2021-11-29 00:00:00+00:00    146
2021-11-30 00:00:00+00:00    207
2021-12-01 00:00:00+00:00    270
Name: AnzahlFall, Length: 632, dtype: int64